In [31]:
import os
%pwd

'C:\\Users\\abdus samad\\Desktop\\Churn_ML\\Churn_ML_Project'

In [32]:
os.chdir("C:/Users/abdus samad/Desktop/Churn_ML/Churn_ML_Project")
%pwd

'C:\\Users\\abdus samad\\Desktop\\Churn_ML\\Churn_ML_Project'

In [33]:
from dataclasses import dataclass
from pathlib import Path


In [34]:
@dataclass 
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str

In [35]:
from src.Churn_Predictor.constants import *
from src.Churn_Predictor.utils.common import read_yaml, create_directories, save_json

In [64]:
class ConfigurationManager:
    def __init__(self, 
                 config_file_path = CONFIG_FILE_PATH, 
                 params_file_path = PARAMS_FILE_PATH):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        self.schema = read_yaml(SCHEMA_FILE_PATH)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.XGBBoost
        target_column = list(self.schema.TARGET_COLUMN.keys())[0]
        
        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path=config.model_path,
            all_params=params,
            metric_file_name=Path(config.metric_file_name),
            target_column=target_column,
            mlflow_uri=os.getenv("MLFLOW_TRACKING_URI")
        )
        return model_evaluation_config

In [ ]:
import os
import pandas as pd
import joblib
import mlflow
import mlflow.sklearn
from dotenv import load_dotenv
from urllib.parse import urlparse
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from src.Churn_Predictor import logger
from src.Churn_Predictor.utils.common import save_json

# Load environment variables at module level
load_dotenv()


class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def evaluate_model(self, actual, pred):
        """Calculate evaluation metrics"""
        accuracy = accuracy_score(actual, pred)
        precision = precision_score(actual, pred)
        recall = recall_score(actual, pred)
        f1 = f1_score(actual, pred)

        return {
            "accuracy": accuracy,
            "precision": precision,
            "recall": recall,
            "f1_score": f1
        }
    
    def setup_mlflow_auth(self):
        """Setup MLflow authentication from environment variables"""
        username = os.getenv('MLFLOW_TRACKING_USERNAME')
        password = os.getenv('MLFLOW_TRACKING_PASSWORD')
        
        if not username or not password:
            logger.warning("MLflow credentials not found in environment. Using local tracking only.")
            return False
        
        # Set credentials as environment variables for MLflow
        os.environ['MLFLOW_TRACKING_USERNAME'] = username
        os.environ['MLFLOW_TRACKING_PASSWORD'] = password
        
        logger.info(f"MLflow authentication configured for user: {username}")
        return True
    
    def initiate_model_evaluation(self):
        """Evaluate model and log to MLflow"""
        logger.info("Starting model evaluation")
        
        # Load test data and model
        logger.info(f"Loading test data from: {self.config.test_data_path}")
        test_data = pd.read_csv(self.config.test_data_path)
        
        logger.info(f"Loading model from: {self.config.model_path}")
        model = joblib.load(self.config.model_path)

        # Prepare test data
        test_x = test_data.drop(self.config.target_column, axis=1)
        test_y = test_data[self.config.target_column]
        
        logger.info(f"Test data shape: {test_x.shape}")
        logger.info(f"Target column: {self.config.target_column}")

        # Setup MLflow authentication
        auth_success = self.setup_mlflow_auth()
        
        # Set tracking and registry URI
        logger.info(f"Setting MLflow tracking URI: {self.config.mlflow_uri}")
        mlflow.set_tracking_uri(self.config.mlflow_uri)
        mlflow.set_registry_uri(self.config.mlflow_uri)
        
        # Set experiment
        mlflow.set_experiment("Churn_Prediction_Model_Evaluation")
        
        # Get tracking URL type
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        try:
            with mlflow.start_run():
                logger.info("MLflow run started")
                
                # Make predictions
                pred = model.predict(test_x)
                
                # Calculate metrics
                metrics = self.evaluate_model(test_y, pred)
                
                logger.info("=" * 50)
                logger.info("MODEL EVALUATION METRICS")
                logger.info("=" * 50)
                logger.info(f"Accuracy:  {metrics['accuracy']:.4f}")
                logger.info(f"Precision: {metrics['precision']:.4f}")
                logger.info(f"Recall:    {metrics['recall']:.4f}")
                logger.info(f"F1-Score:  {metrics['f1_score']:.4f}")
                logger.info("=" * 50)

                # Log parameters and metrics to MLflow
                mlflow.log_params(self.config.all_params)
                mlflow.log_metrics(metrics)
                
                # Save metrics locally
                save_json(path=Path(self.config.metric_file_name), data=metrics)
                logger.info(f"Metrics saved to: {self.config.metric_file_name}")

                # Log model to MLflow
                if tracking_url_type_store != "file":
                    # Remote tracking (DagsHub)
                    logger.info("Logging model to remote MLflow server")
                    mlflow.sklearn.log_model(
                        model, 
                        "model",
                        registered_model_name="Churn_Prediction_Model"
                    )
                else:
                    # Local tracking
                    logger.info("Logging model to local MLflow")
                    mlflow.sklearn.log_model(model, "model")
                
                logger.info("Model evaluation completed successfully")
                
        except Exception as e:
            logger.error(f"Error during MLflow tracking: {e}")
            logger.warning("Continuing without MLflow tracking")
            
            # Save metrics locally even if MLflow fails
            metrics = self.evaluate_model(test_y, model.predict(test_x))
            save_json(path=self.config.metric_file_name, data=metrics)
            logger.info(f"Metrics saved locally to: {self.config.metric_file_name}")
            
            raise e

In [62]:
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

try:
    logger.info("Starting Model Evaluation Pipeline")
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(config=model_evaluation_config)
    model_evaluation.initiate_model_evaluation()
    logger.info("Model evaluation completed successfully")
except Exception as e:
    logger.exception(f"Error in model evaluation: {e}")
    raise e

[2026-02-14 16:38:29,261: INFO: 1851394116: Starting Model Evaluation Pipeline]
[2026-02-14 16:38:29,265: INFO: common: yaml file: config\config.yaml loaded successfully]
[2026-02-14 16:38:29,267: INFO: common: yaml file: params.yaml loaded successfully]
[2026-02-14 16:38:29,272: INFO: common: yaml file: schema.yaml loaded successfully]
[2026-02-14 16:38:29,273: INFO: common: created directory at: artifacts]
[2026-02-14 16:38:29,274: INFO: common: created directory at: artifacts/model_evaluation]
[2026-02-14 16:38:29,275: INFO: 2021659470: Starting model evaluation]
[2026-02-14 16:38:29,276: INFO: 2021659470: Loading test data from: artifacts/data_transformation/test.csv]
[2026-02-14 16:38:29,286: INFO: 2021659470: Loading model from: artifacts/model_trainer/model.joblib]
[2026-02-14 16:38:29,291: INFO: 2021659470: Test data shape: (1405, 23)]
[2026-02-14 16:38:29,292: INFO: 2021659470: Target column: Churn]
[2026-02-14 16:38:29,294: INFO: 2021659470: MLflow authentication configured f

2026/02/14 16:38:35 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
c:\Users\abdus samad\Desktop\Churn_ML\Churn_ML_Project\venv\Lib\site-packages\mlflow\models\model.py:1209: FutureWarning: Saving scikit-learn models in the pickle or cloudpickle format requires exercising caution because these formats rely on Python's object serialization mechanism, which can execute arbitrary code during deserialization.The recommended safe alternative is the 'skops' format.
  flavor.save_model(path=local_path, mlflow_model=mlflow_model, **kwargs)
Successfully registered model 'Churn_Prediction_Model'.
2026/02/14 16:39:02 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: Churn_Prediction_Model, version 1
Created version '1' of model 'Churn_Prediction_Model'.


[2026-02-14 16:39:03,229: INFO: 2021659470: Model evaluation completed successfully]
🏃 View run treasured-chimp-34 at: https://dagshub.com/Maazthepal/Churn_ML_Project.mlflow/#/experiments/0/runs/34dc554dd2ae4c4792c755bbd4e323a7
🧪 View experiment at: https://dagshub.com/Maazthepal/Churn_ML_Project.mlflow/#/experiments/0
[2026-02-14 16:39:04,025: INFO: 1851394116: Model evaluation completed successfully]
